In [1]:
# Import Packages
import numpy as np
import pandas as pd

# Load original data in pandas
data_tran = pd.read_json('data2/data_tran.json', orient='records', lines=True)
data_test = pd.read_json('data2/data_test.json', orient='records', lines=True)

n_tran = data_tran.shape[0]
n_test = data_test.shape[0]

# Load feature data in numpy
x_tran_coauthors = np.load("data2/x_tran_coauthors.npy")
x_tran_venue_a   = np.load("data2/x_tran_venue_a.npy")
x_tran_venue_b   = np.load("data2/x_tran_venue_b.npy")
x_tran_text_a    = np.load("data2/x_tran_text_a.npy")
x_tran_text_b    = np.load("data2/x_tran_text_b.npy")

x_test_coauthors = np.load("data2/x_test_coauthors.npy")
x_test_venue_a   = np.load("data2/x_test_venue_a.npy")
x_test_venue_b   = np.load("data2/x_test_venue_b.npy")
x_test_text_a    = np.load("data2/x_test_text_a.npy")
x_test_text_b    = np.load("data2/x_test_text_b.npy")

x_tran_title_doc2vec = np.load('data2/x_tran_title_doc2vec.npy')
x_test_title_doc2vec = np.load('data2/x_test_title_doc2vec.npy')

x_tran_abstract_doc2vec = np.load('data2/x_tran_abstract_doc2vec.npy')
x_test_abstract_doc2vec = np.load('data2/x_test_abstract_doc2vec.npy')

y_tran_basic = np.load("data2/y_tran.npy")
x_tran_basic = np.concatenate((x_tran_coauthors, x_tran_venue_a, x_tran_venue_b, x_tran_text_a, x_tran_text_b), axis=1)
x_test_basic = np.concatenate((x_test_coauthors, x_test_venue_a, x_test_venue_b, x_test_text_a, x_test_text_b), axis=1)

In [2]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train-validation split
idxs = np.random.permutation(n_tran)
inxs_prop = int(0.85 * n_tran)
idxs_tran_indices = idxs[:inxs_prop]
idxs_vald_indices = idxs[inxs_prop:]

x_tran_a = torch.tensor(x_tran_basic[idxs_tran_indices], dtype=torch.float32).to(device)
x_vald_a = torch.tensor(x_tran_basic[idxs_vald_indices], dtype=torch.float32).to(device)
x_test_a = torch.tensor(x_test_basic, dtype=torch.float32).to(device)

x_tran_b = torch.tensor(x_tran_title_doc2vec[idxs_tran_indices], dtype=torch.float32).to(device)
x_vald_b = torch.tensor(x_tran_title_doc2vec[idxs_vald_indices], dtype=torch.float32).to(device)
x_test_b = torch.tensor(x_test_title_doc2vec, dtype=torch.float32).to(device)

x_tran_c = torch.tensor(x_tran_abstract_doc2vec[idxs_tran_indices], dtype=torch.float32).to(device)
x_vald_c = torch.tensor(x_tran_abstract_doc2vec[idxs_vald_indices], dtype=torch.float32).to(device)
x_test_c = torch.tensor(x_test_abstract_doc2vec, dtype=torch.float32).to(device)

y_tran   = torch.tensor(y_tran_basic[idxs_tran_indices], dtype=torch.float32).to(device)
y_vald   = torch.tensor(y_tran_basic[idxs_vald_indices], dtype=torch.float32).to(device)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

class ModelDataset(Dataset):
    def __init__(self, x1, x2, x3, y):
        self.x1 = x1
        self.x2 = x2
        self.x3 = x3
        self.y  = y

    def __len__(self):
        return len(self.x1)

    def __getitem__(self, idx):
        x1 = self.x1[idx].clone().detach().float()
        x2 = self.x2[idx].clone().detach().float()
        x3 = self.x3[idx].clone().detach().float()
        y  = self.y[idx].clone().detach().float()
        return x1, x2, x3, y


In [4]:
dataset_tran = ModelDataset(x_tran_a, x_tran_b, x_tran_c, y_tran)
datalod_tran = DataLoader(dataset_tran, batch_size=32, shuffle=True)

In [5]:
class FNN(nn.Module):

    def __init__(self, input_dim1, input_dim2, input_dim3, output_dim):
        super(FNN, self).__init__()

        self.model1 = nn.Sequential(
            nn.Linear(input_dim1, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 50),
            nn.ReLU()
        )

        self.model2 = nn.Sequential(
            nn.Linear(input_dim2, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 50),
            nn.ReLU()
        )

        self.model3 = nn.Sequential(
            nn.Linear(input_dim3, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 100),
            nn.ReLU()
        )

        self.fc = nn.Linear(200, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x1, x2, x3):
        x1 = self.model1(x1)
        x2 = self.model2(x2)
        x3 = self.model3(x3)

        x = torch.cat((x1, x2, x3), dim=1)

        x = self.fc(x)
        x = self.sigmoid(x)
        return x


In [6]:
input_dim1 = x_tran_a.shape[1]
input_dim2 = x_tran_b.shape[1]
input_dim3 = x_tran_c.shape[1] 
output_dim = y_tran.shape[1] 

model = FNN(input_dim1, input_dim2, input_dim3, output_dim).to(device)
criterion = nn.BCELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.00001)

In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics(pred_label, true_label):
    pred_label = pred_label.int()
    true_label = true_label.int()
    pc = precision_score(true_label.cpu(), pred_label.cpu(), average='macro', zero_division=0)
    rc = recall_score(true_label.cpu(), pred_label.cpu(), average='macro', zero_division=0)
    f1 = f1_score(true_label.cpu(), pred_label.cpu(), average='macro', zero_division=0)
    return pc, rc, f1

In [8]:
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss):
        if self.best_loss is None or train_loss < self.best_loss - self.delta:
            self.best_loss = train_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

early_stopping = EarlyStopping(patience=10, delta=0.001)

In [9]:
# 开始训练
epochs = 50000

for epoch in range(epochs):
    model.train()  # 设置为训练模式

    total_loss = 0  # 累计每个 epoch 的损失

    # 遍历训练数据集
    for batch in datalod_tran:
        x1, x2, x3, y = batch
        x1, x2, x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)

        # 前向传播
        outputs = model(x1, x2, x3)
        loss = criterion(outputs, y.float())

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(datalod_tran)  # 计算平均损失

    # 每 1000 个 epoch 打印训练和验证结果
    if (epoch + 1) % 100 == 0:
        model.eval()  # 设置为评估模式

        with torch.no_grad():
            # 计算训练集指标
            y_tran_pred_prob = model(x_tran_a, x_tran_b, x_tran_c)
            y_tran_pred_labl = (y_tran_pred_prob > 0.5).int()
            tran_pc, tran_rc, tran_f1 = calculate_metrics(y_tran_pred_labl, y_tran)

            # 计算验证集指标
            y_vald_pred_prob = model(x_vald_a, x_vald_b, x_vald_c)
            y_vald_pred_labl = (y_vald_pred_prob > 0.5).int()
            vald_pc, vald_rc, vald_f1 = calculate_metrics(y_vald_pred_labl, y_vald)

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")
        print(f"Train - Precision: {tran_pc:.4f}, Recall: {tran_rc:.4f}, F1 Score: {tran_f1:.4f}")
        print(f"Val   - Precision: {vald_pc:.4f}, Recall: {vald_rc:.4f}, F1 Score: {vald_f1:.4f}")
        print()

        # 检查早停条件
        early_stopping(avg_loss)
        if early_stopping.early_stop:
            print("Early Stop!")
            break

# 测试集预测（可选）
with torch.no_grad():
    y_test_pred_prob = model(x_test_a, x_test_b, x_test_c)
    y_test_pred_labl = (y_test_pred_prob > 0.5).int()

Epoch 100/50000, Loss: 0.0199
Train - Precision: 0.8983, Recall: 0.5439, F1 Score: 0.6534
Val   - Precision: 0.8075, Recall: 0.4590, F1 Score: 0.5623

Epoch 200/50000, Loss: 0.0098
Train - Precision: 0.9503, Recall: 0.8083, F1 Score: 0.8674
Val   - Precision: 0.8652, Recall: 0.6182, F1 Score: 0.7044

Epoch 300/50000, Loss: 0.0052
Train - Precision: 0.9805, Recall: 0.9373, F1 Score: 0.9576
Val   - Precision: 0.8671, Recall: 0.6771, F1 Score: 0.7478

Epoch 400/50000, Loss: 0.0028
Train - Precision: 0.9963, Recall: 0.9885, F1 Score: 0.9923
Val   - Precision: 0.8686, Recall: 0.6952, F1 Score: 0.7606

Epoch 500/50000, Loss: 0.0016
Train - Precision: 0.9994, Recall: 0.9991, F1 Score: 0.9993
Val   - Precision: 0.8757, Recall: 0.7002, F1 Score: 0.7674

Epoch 600/50000, Loss: 0.0010
Train - Precision: 1.0000, Recall: 0.9999, F1 Score: 0.9999
Val   - Precision: 0.8694, Recall: 0.7068, F1 Score: 0.7695

Epoch 700/50000, Loss: 0.0008
Train - Precision: 1.0000, Recall: 0.9999, F1 Score: 0.9999
Val 

In [10]:
def generate_output_csv(x_test_a, y_test_pred_labl):
    
    result = []
    
    for i, row in enumerate(y_test_pred_labl):
        if ((x_test_a[i, :100] < 1).all() or (x_test_a[i, 100:200] == 0).all() or (x_test_a[i, 200:300] == 0).all() or (x_test_a[i, 300:400] == 0).all() or (x_test_a[i, 400:500] == 0).all()):
            result.append("-1")
        elif row.sum() == 0 or row[100] == 1:
            result.append("-1")
        else:
            indices = [str(idx) for idx, val in enumerate(row) if val == 1]
            result.append(" ".join(indices))
    
    result_df = pd.DataFrame({"ID": range(len(result)), "Predict": result})
    
    return result_df

generate_output_csv(x_test_a, y_test_pred_labl).to_csv("result_method2.csv", index=False)